In [6]:
# Iniciando as bibliotecas usadas para montar um sistema de aprendizado de máquina chamado árvore de decisão
# No qual tem como objetivo criar um "raciocínio" das informações de saída e entrada e fornecer um método no qual seja possivel prever informações 
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#GRU - EXEMPLO - ESTUDAR 

In [7]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

Tensorflow/Keras: 2.8.0
pandas: 1.3.5
numpy: 1.21.6
sklearn: 1.0.2
plotly: 5.5.0


In [8]:
# Set Pandas options to display more columns
pd.options.display.max_columns=150

In [9]:
# Read in the weather data csv - keep only the columns we need
#df=pd.read_csv('/content/drive/MyDrive/Análise Artur/weatherAUS.csv', encoding='utf-8', usecols=['Date', 'Location', 'MinTemp', 'MaxTemp'])
df=pd.read_excel('/content/drive/MyDrive/Artur/dados_vazao_precipitacao_preenchido_v4.xlsx')

In [10]:
# Drop records where target MinTemp=NaN or MaxTemp=NaN
# df=df[pd.isnull(df['MinTemp'])==False]
# df=df[pd.isnull(df['MaxTemp'])==False]

In [11]:
# Convert dates to year-months
# df['Year-Month']= (pd.to_datetime(df['Date'], yearfirst=True)).dt.strftime('%Y-%m')

In [12]:
# Derive median daily temperature (mid point between Daily Max and Daily Min)
# df['MedTemp']=df[['MinTemp', 'MaxTemp']].median(axis=1)
df = df.transpose()

In [13]:
# Show a snaphsot of data
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,...,3176,3177,3178,3179,3180,3181,3182,3183,3184,3185,3186,3187,3188,3189,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199,3200,3201,3202,3203,3204,3205,3206,3207,3208,3209,3210,3211,3212,3213,3214,3215,3216,3217,3218,3219,3220,3221,3222,3223,3224,3225,3226,3227,3228,3229,3230,3231,3232,3233,3234,3235,3236,3237,3238,3239,3240,3241,3242,3243,3244,3245,3246,3247,3248,3249,3250
datat0,2012-07-15 00:00:00,2012-07-16 00:00:00,2012-07-17 00:00:00,2012-07-18 00:00:00,2012-07-19 00:00:00,2012-07-20 00:00:00,2012-07-21 00:00:00,2012-07-22 00:00:00,2012-07-23 00:00:00,2012-07-24 00:00:00,2012-07-25 00:00:00,2012-07-26 00:00:00,2012-07-27 00:00:00,2012-07-28 00:00:00,2012-07-29 00:00:00,2012-07-30 00:00:00,2012-07-31 00:00:00,2012-08-01 00:00:00,2012-08-02 00:00:00,2012-08-03 00:00:00,2012-08-04 00:00:00,2012-08-05 00:00:00,2012-08-06 00:00:00,2012-08-07 00:00:00,2012-08-08 00:00:00,2012-08-09 00:00:00,2012-08-10 00:00:00,2012-08-11 00:00:00,2012-08-12 00:00:00,2012-08-13 00:00:00,2012-08-14 00:00:00,2012-08-15 00:00:00,2012-08-16 00:00:00,2012-08-17 00:00:00,2012-08-18 00:00:00,2012-08-19 00:00:00,2012-08-20 00:00:00,2012-08-21 00:00:00,2012-08-22 00:00:00,2012-08-23 00:00:00,2012-08-24 00:00:00,2012-08-25 00:00:00,2012-08-26 00:00:00,2012-08-27 00:00:00,2012-08-28 00:00:00,2012-08-29 00:00:00,2012-08-30 00:00:00,2012-08-31 00:00:00,2012-09-01 00:00:00,2012-09-02 00:00:00,2012-09-03 00:00:00,2012-09-04 00:00:00,2012-09-05 00:00:00,2012-09-06 00:00:00,2012-09-07 00:00:00,2012-09-08 00:00:00,2012-09-09 00:00:00,2012-09-10 00:00:00,2012-09-11 00:00:00,2012-09-12 00:00:00,2012-09-13 00:00:00,2012-09-14 00:00:00,2012-09-15 00:00:00,2012-09-16 00:00:00,2012-09-17 00:00:00,2012-09-18 00:00:00,2012-09-19 00:00:00,2012-09-20 00:00:00,2012-09-21 00:00:00,2012-09-22 00:00:00,2012-09-23 00:00:00,2012-09-24 00:00:00,2012-09-25 00:00:00,2012-09-26 00:00:00,2012-09-27 00:00:00,...,2021-03-26 00:00:00,2021-03-27 00:00:00,2021-03-28 00:00:00,2021-03-29 00:00:00,2021-03-30 00:00:00,2021-03-31 00:00:00,2021-04-01 00:00:00,2021-04-02 00:00:00,2021-04-03 00:00:00,2021-04-04 00:00:00,2021-04-05 00:00:00,2021-04-06 00:00:00,2021-04-07 00:00:00,2021-04-08 00:00:00,2021-04-09 00:00:00,2021-04-10 00:00:00,2021-04-11 00:00:00,2021-04-12 00:00:00,2021-04-13 00:00:00,2021-04-14 00:00:00,2021-04-15 00:00:00,2021-04-16 00:00:00,2021-04-17 00:00:00,2021-04-18 00:00:00,2021-04-19 00:00:00,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00,2021-04-26 00:00:00,2021-04-27 00:00:00,2021-04-28 00:00:00,2021-04-29 00:00:00,2021-04-30 00:00:00,2021-05-01 00:00:00,2021-05-02 00:00:00,2021-05-03 00:00:00,2021-05-04 00:00:00,2021-05-05 00:00:00,2021-05-06 00:00:00,2021-05-07 00:00:00,2021-05-08 00:00:00,2021-05-09 00:00:00,2021-05-10 00:00:00,2021-05-11 00:00:00,2021-05-12 00:00:00,2021-05-13 00:00:00,2021-05-14 00:00:00,2021-05-15 00:00:00,2021-05-16 00:00:00,2021-05-17 00:00:00,2021-05-18 00:00:00,2021-05-19 00:00:00,2021-05-20 00:00:00,2021-05-21 00:00:00,2021-05-22 00:00:00,2021-05-23 00:00:00,2021-05-24 00:00:00,2021-05-25 00:00:00,2021-05-26 00:00:00,2021-05-27 00:00:00,2021-05-28 00:00:00,2021-05-29 00:00:00,2021-05-30 00:00:00,2021-05-31 00:00:00,2021-06-01 00:00:00,2021-06-02 00:00:00,2021-06-03 00:00:00,2021-06-04 00:00:00,2021-06-05 00:00:00,2021-06-06 00:00:00,2021-06-07 00:00:00,2021-06-08 00:00:00
vazaot0,3.065,2.8985,2.5836,1.897,1.5496,1.4439,1.4439,1.343,1.343,1.2469,1.343,1.343,1.5496,1.5496,1.343,1.343,1.343,1.343,1.343,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.4439,1.5496,1.4439,1.5496,1.897,1.5496,1.776,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.2469,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1

In [14]:
# Create a copy of an original dataframe
# df2=df[['Location', 'Year-Month', 'MedTemp']].copy()
#df2 = df.copy()
# Calculate monthly average temperature for each location
# df2=df2.groupby(['Location', 'Year-Month'], as_index=False).mean()

# Transpose dataframe 
# df2_pivot=df2.pivot(index=['Location'], columns='Year-Month')['MedTemp']
#df2_pivot = df2.pivot(index=['p1t0'], columns='datat0') #['MedTemp']

# Remove locations with lots of missing (NaN) data
# df2_pivot=df2_pivot.drop(['Dartmoor', 'Katherine', 'Melbourne', 'Nhil', 'Uluru'], axis=0)

# Remove months with lots of missing (NaN) data
# df2_pivot=df2_pivot.drop(['2007-11', '2007-12', '2008-01', '2008-02', '2008-03', '2008-04', '2008-05', '2008-06', '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06'], axis=1)

df2_pivot = df.iloc[pd.RangeIndex(len(df)).drop(0)]
del df
# Display the new dataframe
df2_pivot

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,...,3176,3177,3178,3179,3180,3181,3182,3183,3184,3185,3186,3187,3188,3189,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199,3200,3201,3202,3203,3204,3205,3206,3207,3208,3209,3210,3211,3212,3213,3214,3215,3216,3217,3218,3219,3220,3221,3222,3223,3224,3225,3226,3227,3228,3229,3230,3231,3232,3233,3234,3235,3236,3237,3238,3239,3240,3241,3242,3243,3244,3245,3246,3247,3248,3249,3250
vazaot0,3.065,2.8985,2.5836,1.897,1.5496,1.4439,1.4439,1.343,1.343,1.2469,1.343,1.343,1.5496,1.5496,1.343,1.343,1.343,1.343,1.343,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.4439,1.5496,1.4439,1.5496,1.897,1.5496,1.776,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.2469,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.1555,0.9861,0.834,0.834,0.834,0.834,0.9861,1.0686,0.834,0.7641,0.7641,0.6981,0.5778,0.5778,0.5778,...,0.0,0.0,0.0,0.5946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.1918,10.6336,0.4105,0.0217,0.5946,1.5416,0.0814,0.0217,0.0575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0027,13.2212,20.1392,3.4964,0.2159,0.0814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9728,0.3058,0.141,10.6336,1.9891,0.0814,0.01,0.0,0.0,0.0027,0.0027,0.0
vazaot1,1.343,1.343,1.2469,1.343,1.343,1.5496,1.5496,1.343,1.343,1.343,1.343,1.343,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.4439,1.5496,1.4439,1.5496,1.897,1.5496,1.776,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.2469,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.1555,0.9861,0.834,0.834,0.834,0.834,0.9861,1.0686,0.834,0.7641,0.7641,0.6981,0.5778,0.5778,0.5778,0.5778,0.5231,0.472,0.472,0.472,0.5231,0.5231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.1918,10.6336,0.4105,0.0217,0.5946,1.5416,0.0814,0.0217,0.0575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0027,13.2212,20.1392,3.4964,0.2159,0.0814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9728,0.3058,0.141,10.6336,1.9891,0.0814,0.01,0.0,0.0,0.0027,0.0027,0.0,0.0027,0.0,0.0,0.0,0.0,0.0,0.0
vazaot2,1.343,1.343,1.343,1.343,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.4439,1.5496,1.4439,1.5496,1.897,1.5496,1.776,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.2469,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.1555,0.9861,0.834,0.834,0.834,0.834,0.9861,1.0686,0.834,0.7641,0.7641,0.6981,0.5778,0.5778,0.5778,0.5778,0.5231,0.472,0.472,0.472,0.5231,0.5231,0.4242,0.472,0.5231,0.6981,0.7641,0.9861,1.4439,1.776,...,0.0,11.1918,10.6336,0.4105,0.0217,0.5946,1.5416,0.0814,0.0217,0.0575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0027,13.2212,20.1392,3.4964,0.2159,0.0814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9728,0.3058,0.141,10.6336,1.9891,0.0814,0.01,0.0,0.0,0.0027,0.0027,0.0,0.0027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vazaot3,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.4439,1.5496,1.4439,1.5496,1.897,1.5496,1.776,1.5496,1.343,1.343,1.343,1.343,1.343,1.343,1.343,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.2469,1.2469,1.1555,1.1555,1.1555,1.0686,1.0686,1.1555,1.1555,0.9861,0.834,0.834,0.834,0.834,0.9861,1.0686,0.834,0.7641,0.7641,0.6981,0.5778,0.5778,0.5778,0.5778,0.5231,0.472,0.472,0.472,0.5231,0.5231,0.4242,0.472,0.5231,0.6981,0.7641,0.9861,1.4439,1.776,1.343,1.2469,1.1555,0.9861,0.834,0.834,0.7641,...,0.0814,0.0217,0.0575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0027,13.2212,20.1392,3.4964,0.2159,0.0814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9728,0.3058,0.141,10.6336,1.9891,0.0814,0.01,0.0,0.0,0.0027,0.002

In [15]:
# Add missing months 2011-04, 2011-04, 2011-04 and impute data
#df2_pivot['2011-04']=(df2_pivot['2011-03']+df2_pivot['2011-05'])/2
#df2_pivot['2012-12']=(df2_pivot['2012-11']+df2_pivot['2013-01'])/2
#df2_pivot['2013-02']=(df2_pivot['2013-01']+df2_pivot['2013-03'])/2

# Sort columns so Year-Months are in the correct order
df2_pivot=df2_pivot.reindex(sorted(df2_pivot.columns), axis=1)

In [16]:
# Plot average monthly temperature derived from daily medians for each location
fig = go.Figure()
for datat0 in df2_pivot.index:
    fig.add_trace(go.Scatter(x=df2_pivot.loc[datat0, :].index, 
                             y=df2_pivot.loc[datat0, :].values,
                             mode='lines',
                             name=datat0,
                             opacity=0.8,
                             line=dict(width=1)
                            ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'), showlegend=True)

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title=''
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title=''
                )

# Set figure title
fig.update_layout(title=dict(text="Dados recebidos", font=dict(color='black')))

fig.show()

In [17]:
def shaping(datain, timestep, scaler):
    
    # Loop through each location
    for datat0 in datain.index:
        datatmp = datain[datain.index==datat0].copy()
    
        # Convert input dataframe to array and flatten
        arr=datatmp.to_numpy().flatten() 
        
        # Scale using transform (using previously fitted scaler)
        arr_scaled=scaler.transform(arr.reshape(-1, 1)).flatten()
        
        cnt=0
        for mth in range(0, len(datatmp.columns)-(2*timestep)+1): # Define range 
            cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
            X_start=mth # Start month for inputs of each sample
            X_end=mth+timestep # End month for inputs of each sample
            Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
            Y_end=mth+2*timestep # End month for targets of each sample.  

            # Assemble input and target arrays containing all samples
            if mth==0:
                X_comb=arr_scaled[X_start:X_end]
                Y_comb=arr_scaled[Y_start:Y_end]
            else: 
                X_comb=np.append(X_comb, arr_scaled[X_start:X_end])
                Y_comb=np.append(Y_comb, arr_scaled[Y_start:Y_end])

        # Reshape input and target arrays 
        X_loc=np.reshape(X_comb, (cnt, timestep, 1))
        Y_loc=np.reshape(Y_comb, (cnt, timestep, 1))
        
        # Append an array for each location to the master array
        if datat0==datain.index[0]:
            X_out=X_loc
            Y_out=Y_loc
        else:
            X_out=np.concatenate((X_out, X_loc), axis=0)
            Y_out=np.concatenate((Y_out, Y_loc), axis=0)
            
    return X_out, Y_out

In [ ]:
##### Step 1 - Specify parameters
timestep=18
scaler = MinMaxScaler(feature_range=(-1, 1))


##### Step 2 - Prepare data

# Split data into train and test dataframes
df_train=df2_pivot.iloc[:, 0:-2*timestep].copy()
df_test=df2_pivot.iloc[:, -2*timestep:].copy()

# Use fit to train the scaler on the training data only, actual scaling will be done inside reshaping function
scaler.fit(df_train.to_numpy().reshape(-1, 1))

# Use previously defined shaping function to reshape the data for GRU
X_train, Y_train = shaping(datain=df_train, timestep=timestep, scaler=scaler)
X_test, Y_test = shaping(datain=df_test, timestep=timestep, scaler=scaler)


##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="GRU-Model") # Model
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-GRU-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(X_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-GRU-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x


##### Step 4 - Compile the model
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )


##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=50, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=1, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=10, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                   )


##### Step 6 - Use model to make predictions
# Predict results on training data
#pred_train = model.predict(X_train)
# Predict results on test data
pred_test = model.predict(X_test)


##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")

Epoch 1/50
10176/10176 [==============================] - 147s 14ms/step - loss: 0.0070 - mean_squared_error: 0.0070 - mean_absolute_error: 0.0358
Epoch 2/50
10176/10176 [==============================] - 142s 14ms/step - loss: 0.0060 - mean_squared_error: 0.0060 - mean_absolute_error: 0.0308
Epoch 3/50
10176/10176 [==============================] - 142s 14ms/step - loss: 0.0059 - mean_squared_error: 0.0059 - mean_absolute_error: 0.0293
Epoch 4/50
10176/10176 [==============================] - 140s 14ms/step - loss: 0.0059 - mean_squared_error: 0.0059 - mean_absolute_error: 0.0286
Epoch 5/50
10176/10176 [==============================] - 142s 14ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - mean_absolute_error: 0.0284
Epoch 6/50
10176/10176 [==============================] - 142s 14ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - mean_absolute_error: 0.0279
Epoch 7/50
10176/10176 [==============================] - 143s 14ms/step - loss: 0.0057 - mean_squared_error: 0.0057 -

In [ ]:
# Select locations to predict temperatures for
datat0=['vazaot0','vazaot1','vazaot2','vazaot3']
dfloc_test = df_test[df_test.index.isin(datat0)].copy()

# Reshape test data
X_test, Y_test = shaping(datain=dfloc_test, timestep=timestep, scaler=scaler)

# Predict results on test data
pred_test = model.predict(X_test)

In [ ]:
# Plot average monthly temperatures (actual and predicted) for test (out of time) data
fig = go.Figure()

# Trace for actual temperatures
for datat0 in dfloc_test.index:
    fig.add_trace(go.Scatter(x=dfloc_test.loc[datat0, :].index, 
                             y=dfloc_test.loc[datat0, :].values,
                             mode='lines',
                             name=datat0,
                             opacity=0.8,
                             line=dict(width=1)
                            ))

# Trace for predicted temperatures
for i in range(0,pred_test.shape[0]):
    fig.add_trace(go.Scatter(x=np.array(dfloc_test.columns[-timestep:]),
                             # Need to inverse transform the predictions before plotting
                             y=scaler.inverse_transform(pred_test[i].reshape(-1,1)).flatten(), 
                             mode='lines', 
                             name=dfloc_test.index[i]+' Prediction',
                             opacity=1,
                             line=dict(width=2, dash='dot')
                            ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Year-Month'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='l/s'
                )

# Set figure title
fig.update_layout(title=dict(text="predição dos dados de vazões 7, 14 e 21 dias
", font=dict(color='black')))
fig.show()

SyntaxError: ignored

#GRU - EXEMPLO - ESTUDAR 

In [ ]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

In [ ]:
# Set Pandas options to display more columns
pd.options.display.max_columns=150

In [ ]:
# Read in the weather data csv - keep only the columns we need
#df=pd.read_csv('/content/drive/MyDrive/Análise Artur/weatherAUS.csv', encoding='utf-8', usecols=['Date', 'Location', 'MinTemp', 'MaxTemp'])
df=pd.read_excel('/content/drive/MyDrive/Análise Artur/dados_vazao_precipitacao_preenchido_v3.xlsx')

In [ ]:
# Drop records where target MinTemp=NaN or MaxTemp=NaN
# df=df[pd.isnull(df['MinTemp'])==False]
# df=df[pd.isnull(df['MaxTemp'])==False]

In [ ]:
# Convert dates to year-months
# df['Year-Month']= (pd.to_datetime(df['Date'], yearfirst=True)).dt.strftime('%Y-%m')

In [ ]:
# Derive median daily temperature (mid point between Daily Max and Daily Min)
# df['MedTemp']=df[['MinTemp', 'MaxTemp']].median(axis=1)
df = df.transpose()

In [ ]:
# Show a snaphsot of data
df

In [ ]:
# Create a copy of an original dataframe
# df2=df[['Location', 'Year-Month', 'MedTemp']].copy()
#df2 = df.copy()
# Calculate monthly average temperature for each location
# df2=df2.groupby(['Location', 'Year-Month'], as_index=False).mean()

# Transpose dataframe 
# df2_pivot=df2.pivot(index=['Location'], columns='Year-Month')['MedTemp']
#df2_pivot = df2.pivot(index=['p1t0'], columns='datat0') #['MedTemp']

# Remove locations with lots of missing (NaN) data
# df2_pivot=df2_pivot.drop(['Dartmoor', 'Katherine', 'Melbourne', 'Nhil', 'Uluru'], axis=0)

# Remove months with lots of missing (NaN) data
# df2_pivot=df2_pivot.drop(['2007-11', '2007-12', '2008-01', '2008-02', '2008-03', '2008-04', '2008-05', '2008-06', '2008-07', '2008-08', '2008-09', '2008-10', '2008-11', '2008-12', '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06'], axis=1)

df2_pivot = df.iloc[pd.RangeIndex(len(df)).drop(0)]
del df
# Display the new dataframe
df2_pivot

In [ ]:
# Add missing months 2011-04, 2011-04, 2011-04 and impute data
#df2_pivot['2011-04']=(df2_pivot['2011-03']+df2_pivot['2011-05'])/2
#df2_pivot['2012-12']=(df2_pivot['2012-11']+df2_pivot['2013-01'])/2
#df2_pivot['2013-02']=(df2_pivot['2013-01']+df2_pivot['2013-03'])/2

# Sort columns so Year-Months are in the correct order
df2_pivot=df2_pivot.reindex(sorted(df2_pivot.columns), axis=1)

In [ ]:
# Plot average monthly temperature derived from daily medians for each location
fig = go.Figure()
for datat0 in df2_pivot.index:
    fig.add_trace(go.Scatter(x=df2_pivot.loc[datat0, :].index, 
                             y=df2_pivot.loc[datat0, :].values,
                             mode='lines',
                             name=datat0,
                             opacity=0.8,
                             line=dict(width=1)
                            ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'), showlegend=True)

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title=''
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title=''
                )

# Set figure title
fig.update_layout(title=dict(text="Dados recebidos", font=dict(color='black')))

fig.show()

In [ ]:
def shaping(datain, timestep, scaler):
    
    # Loop through each location
    for datat0 in datain.index:
        datatmp = datain[datain.index==datat0].copy()
    
        # Convert input dataframe to array and flatten
        arr=datatmp.to_numpy().flatten() 
        
        # Scale using transform (using previously fitted scaler)
        arr_scaled=scaler.transform(arr.reshape(-1, 1)).flatten()
        
        cnt=0
        for mth in range(0, len(datatmp.columns)-(2*timestep)+1): # Define range 
            cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
            X_start=mth # Start month for inputs of each sample
            X_end=mth+timestep # End month for inputs of each sample
            Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
            Y_end=mth+2*timestep # End month for targets of each sample.  

            # Assemble input and target arrays containing all samples
            if mth==0:
                X_comb=arr_scaled[X_start:X_end]
                Y_comb=arr_scaled[Y_start:Y_end]
            else: 
                X_comb=np.append(X_comb, arr_scaled[X_start:X_end])
                Y_comb=np.append(Y_comb, arr_scaled[Y_start:Y_end])

        # Reshape input and target arrays 
        X_loc=np.reshape(X_comb, (cnt, timestep, 1))
        Y_loc=np.reshape(Y_comb, (cnt, timestep, 1))
        
        # Append an array for each location to the master array
        if datat0==datain.index[0]:
            X_out=X_loc
            Y_out=Y_loc
        else:
            X_out=np.concatenate((X_out, X_loc), axis=0)
            Y_out=np.concatenate((Y_out, Y_loc), axis=0)
            
    return X_out, Y_out

In [ ]:
##### Step 1 - Specify parameters
timestep=18
scaler = MinMaxScaler(feature_range=(-1, 1))


##### Step 2 - Prepare data

# Split data into train and test dataframes
df_train=df2_pivot.iloc[:, 0:-2*timestep].copy()
df_test=df2_pivot.iloc[:, -2*timestep:].copy()

# Use fit to train the scaler on the training data only, actual scaling will be done inside reshaping function
scaler.fit(df_train.to_numpy().reshape(-1, 1))

# Use previously defined shaping function to reshape the data for GRU
X_train, Y_train = shaping(datain=df_train, timestep=timestep, scaler=scaler)
X_test, Y_test = shaping(datain=df_test, timestep=timestep, scaler=scaler)


##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="GRU-Model") # Model
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-GRU-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(X_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-GRU-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x


##### Step 4 - Compile the model
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )


##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=50, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=1, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=10, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                   )


##### Step 6 - Use model to make predictions
# Predict results on training data
#pred_train = model.predict(X_train)
# Predict results on test data
pred_test = model.predict(X_test)


##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")

In [ ]:
# Select locations to predict temperatures for
datat0=['vazaot0','vazaot1','vazaot2','vazaot3']
dfloc_test = df_test[df_test.index.isin(datat0)].copy()

# Reshape test data
X_test, Y_test = shaping(datain=dfloc_test, timestep=timestep, scaler=scaler)

# Predict results on test data
pred_test = model.predict(X_test)

In [ ]:
# Plot average monthly temperatures (actual and predicted) for test (out of time) data
fig = go.Figure()

# Trace for actual temperatures
for datat0 in dfloc_test.index:
    fig.add_trace(go.Scatter(x=dfloc_test.loc[datat0, :].index, 
                             y=dfloc_test.loc[datat0, :].values,
                             mode='lines',
                             name=datat0,
                             opacity=0.8,
                             line=dict(width=1)
                            ))

# Trace for predicted temperatures
for i in range(0,pred_test.shape[0]):
    fig.add_trace(go.Scatter(x=np.array(dfloc_test.columns[-timestep:]),
                             # Need to inverse transform the predictions before plotting
                             y=scaler.inverse_transform(pred_test[i].reshape(-1,1)).flatten(), 
                             mode='lines', 
                             name=dfloc_test.index[i]+' Prediction',
                             opacity=1,
                             line=dict(width=2, dash='dot')
                            ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Year-Month'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Average Monthly Temperatures", font=dict(color='black')))
fig.show()